In [ ]:
#필요한 것들 install
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U
!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-gpu
!pip install langchain-teddynote
!pip install peft

In [ ]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF

import random
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    set_seed
)

# 시드 설정
seed = 42
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
set_seed(seed)

from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

import subprocess
from IPython.display import FileLink, display
from huggingface_hub import login
import wandb

torch.utils.checkpoint.use_reentrant = False
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="bitsandbytes")

In [ ]:
test_source = [
    "「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》",
    "「FIS 이슈 & 포커스」 23-2호 《핵심재정사업 성과관리》",
    "「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》",
    "「FIS 이슈&포커스」 22-2호 《재정성과관리제도》",
    "국토교통부_행복주택출자",
    "보건복지부_노인장기요양보험 사업운영",
    "보건복지부_부모급여(영아수당) 지원",
    "산업통상자원부_에너지바우처",
    "중소벤처기업부_혁신창업사업화자금(융자)"
]

train_source = [
    "「FIS 이슈 & 포커스」 22-3호 《재정융자사업》",
    "「FIS 이슈 & 포커스」 23-3호 《조세지출 연계관리》",
    "1-1 2024 주요 재정통계 1권",
    "2024 나라살림 예산개요",
    "2024년도 성과계획서(총괄편)",
    "고용노동부_내일배움카드(일반)",
    "고용노동부_조기재취업수당",
    "고용노동부_청년일자리창출지원",
    "국토교통부_민간임대(융자)",
    "국토교통부_소규모주택정비사업",
    "국토교통부_전세임대(융자)",
    "보건복지부_노인일자리 및 사회활동지원",
    "보건복지부_생계급여",
    "월간 나라재정 2023년 12월호",
    "재정통계해설",
    "중소벤처기업부_창업사업화지원"
]
# 두 리스트를 합칩니다
all_sources = test_source + train_source

# 딕셔너리를 생성합니다
h2n = {name : str(idx) for idx, name in enumerate(all_sources)}
n2h = {str(idx) : name for idx, name in enumerate(all_sources)}

for i in range(len(all_sources)):
    print(i,n2h[str(i)])


In [ ]:
#선택할 옵션들, 여기를 수정하세요
class Opt:
    def __init__(self):
        self.llm_model = "meta-llama/Meta-Llama-3.1-8B-Instruct" #based on yours
        self.embeddings_model = 'intfloat/multilingual-e5-large' #based on yours
        self.base_directory = "/kaggle/input/rag-data" #based on yours
        self.output_dir = "/kaggle/working/results" #based on yours
        os.makedirs(self.output_dir, exist_ok=True) 
        #없으면 디렉토리 생성
        self.hf_token = '' #based on yours
        #Jupyter Notebook에서 작업했기 떄문에 직접 입력 
        
        self.train_csv_path = os.path.join(self.base_directory, "train.csv")
        self.test_csv_path = os.path.join(self.base_directory, "test.csv")
        self.submission_csv_path = os.path.join(self.base_directory, "sample_submission.csv")
        self.results_path = os.path.join(self.output_dir, "submission.csv")
        self.chunk_size = 512
        self.chunk_overlap = 32

        self.template = """
            다음 정보를 바탕으로 질문에 답하세요:
            {context}
            
            ### 질문:
            {question}
            
            ### 답변:
            
            <|eot_id|>
            """
            #<|eot_id|>는 llama3.1 Instruct모델에서 사용한 특수 토큰으로 사용시 답변이 길어지는것을 방지함, 모델이 바뀐다면 확인해야함
args=Opt()

# Hugging Face 로그인 , 미리 해당 모델의 사용 권환을 승인 받아야함
hf_token = args.hf_token
login(hf_token)

# wandb.login(key ='') 
# wandb.init(project="")  # 프로젝트 이름 설정


In [ ]:
def process_pdf(file_path, chunk_size=512, chunk_overlap=32):
    #PDF 텍스트 추출 후 chunk 단위로 나누기
    #PDF 파일 열기
    for i in h2n:
        if i in file_path:
            file_path = file_path.replace(i,h2n[i])
            break
            
    doc = fitz.open(file_path)
    text = ''
    
    for page in doc:
        text += page.get_text()
    
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks

def create_vector_db(chunks, model_path="intfloat/multilingual-e5-large"):
    #FAISS DB생성
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db
    
def normalize_string(string):
    #유니코드 정규화
    return unicodedata.normalize('NFC', string)

def process_pdfs_from_dataframe(df, base_directory):
    #PDF정보로 Retreiver생성
    all_chunks = []
    unique_paths = df['Source_path'].unique()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        normalized_path = normalize_string(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        chunks = process_pdf(full_path,args.chunk_size,args.chunk_overlap)
        all_chunks.extend(chunks)
    
    print("Creating vector database...")
    db = create_vector_db(all_chunks,args.embeddings_model)
    
    retriever = db.as_retriever()
    
    return {
        'db': db,
        'retriever': retriever
    }
    
def get_combined_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
df = pd.read_csv(args.train_csv_path)
pdf_databases = process_pdfs_from_dataframe(df, args.base_directory)

In [ ]:
def setup_llm_pipeline():
    tokenizer = AutoTokenizer.from_pretrained(args.llm_model)
    tokenizer.use_default_system_prompt = False

    num_gpus = torch.cuda.device_count()
    max_memory = {i: "14GiB" for i in range(num_gpus)}
    #멀티 GPU할당 Kaggle T4 x 2 이기 때문에 하나에 14GB할당
    model = AutoModelForCausalLM.from_pretrained(
        args.llm_model,
        quantization_config=BitsAndBytesConfig(
                load_in_8bit=True  # 8-bit 양자화 활성화
            ),
        torch_dtype="auto",
        device_map="balanced",
        max_memory=max_memory,
        trust_remote_code=True
    )
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,
        max_new_tokens=256,
    )

    return HuggingFacePipeline(pipeline=text_generation_pipeline)

llm = setup_llm_pipeline()

In [ ]:
results = []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    source = normalize_string(row['Source'])
    question = row['Question']

    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys['retriever']

    prompt = PromptTemplate.from_template(args.template) 
   
    # RAG 체인 정의
    rag_chain = (
        {"context": retriever | get_combined_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)
    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response

    })

In [ ]:
#임시
import pandas as pd
import os
import subprocess
from IPython.display import FileLink, display, HTML
class Opt():
    def __init__(self):
        self.results_path='/kaggle/working/results/submission.csv'
        self.submission_csv_path = '/kaggle/input/rag-data/sample_submission.csv'
        self.output_dir = '/kaggle/working/results'
        os.makedirs(self.output_dir, exist_ok=True) 

args=Opt()
results = []
for _ in range(98):    # 결과 저장
    results.append({
        "Source": 'a',
        "Source_path": 'a',
        "Question": 'a',
        "Answer": 'a'
    })

In [ ]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv(args.submission_csv_path)

# 생성된 답변에서 앞뒤의 공백 및 줄바꿈 제거 후 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'].strip() for item in results]  # strip()으로 앞뒤 공백 제거
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")  # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [주의]

print(submit_df.head(3))
submit_df.to_csv(args.results_path, encoding='UTF-8-sig', index=False)

In [ ]:
#결과 다운 코드 생성
def download_file(path):
    os.chdir(os.path.dirname(path))
    
    command = f"zip -r output.zip {os.path.basename(path)}/"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink('output.zip'))
    
download_file(args.output_dir)